# Installation

In [ ]:
# pip install -r requirements.txt

# Tool usage test

In [1]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Add console handler only if not already present
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

Persona Extractor

In [3]:
from src.tools.persona_extractor import get_persona_extractor_tool

tool1 = get_persona_extractor_tool('user_41', 'Lifestyle Optimization')

result1 = tool1.invoke({"sentence": "I also like to eat pasta for dinner."})
print(result1)

Device set to use mps:0
INFO - SQLitePersonaDB initialized
INFO - [EXTRACTED] relation='favorite_food', object='pasta', topic='Skill', task='Lifestyle Optimization'
INFO - Persona fact inserted for user user_41, task Lifestyle Optimization
INFO - [SAVED TO DB] for username='user_41'


The persona has extracted. Just inform the user.


Community-based Recommender:

In [1]:
from src.tools.community_recommender import create_community_recommender_tool

tool2 = create_community_recommender_tool(username="user_41", task="Content Consumption")

result2 = tool2.invoke({"sentence": "Can you provide community recommendations?"})
print(result2)

INFO:persona_util:SQLitePersonaDB initialized
INFO:src.tools.community_recommender:[TOOL INVOKED] Community recommender triggered with input: Can you provide community recommendations?


[TOOL RESULT]
Type: Recommendation:
Community suggestions for Content Consumption (from other users):
- Book:
  . reading graphic novels: liked by 5 users
  . reading science fiction: liked by 5 users
  . reading cookbooks: liked by 4 users
  . reading mysteries: liked by 4 users
- Game:
  . card games: liked by 6 users
  . strategy games: liked by 6 users
  . arcade games: liked by 5 users
  . role-playing games: liked by 5 users
- Movie:
  . watching animated movies: liked by 6 users
  . watching fantasy movies: liked by 6 users
  . watching historical dramas: liked by 6 users
  . watching adventure movies: liked by 5 users
- Music:
  . hip hop music: liked by 6 users
  . indie music: liked by 6 users
  . reggae music: liked by 6 users
  . blues music: liked by 5 users


# Agent Testing

In [ ]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [1]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [2]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)


# Add console handler
handler = logging.StreamHandler()
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [3]:
from dotenv import load_dotenv
load_dotenv() 

True

In [4]:
from src.utils.persona_util import SQLitePersonaDB
db = SQLitePersonaDB()

USERNAME = "user_41"

db.create_user(USERNAME, 'Sali A', 28, 'male', 'user')

USER = db.get_user(USERNAME)

task_list = [f"{task['name']}" for task in db.get_all_tasks()]

TASK = task_list[0]
print(TASK)

USER_PERSONA = db.get_user_persona_summary_by_task('user_01', TASK)
USER_PERSONA = USER_PERSONA.replace('user_01', USERNAME)

USER_PERSONA

INFO - SQLitePersonaDB initialized
INFO - User created: user_41


Content Consumption


'User user_41 preferences for Content Consumption: | Book: likes reading novels | Game: plays video games | Movie: likes watching sci-fi | Music: likes classical music'

## OpenAI Agent

In [5]:
from src.agents.PersoAgent import PersoAgent

agent = PersoAgent(
    user = USER,
    task = TASK,
    prev_personas = USER_PERSONA
)

INFO - Initializing PersoAgent for user: {'username': 'user_41', 'full_name': 'Sali A', 'age': 28, 'gender': 'male', 'role': 'user'} with task: Content Consumption
Device set to use mps:0
INFO - SQLitePersonaDB initialized
INFO - SQLitePersonaDB initialized


In [6]:
# Simple invocation - just pass the task
response = agent.handle_task("I also enjoy horror movies.")

print(response)



> Entering new AgentExecutor chain...


INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `PersonaExtractor` with `{'sentence': 'I enjoy horror movies.'}`




INFO - [EXTRACTED] relation='favorite_movie', object='horror', topic='Movie', task='Content Consumption'
INFO - Persona fact inserted for user user_41, task Content Consumption
INFO - [SAVED TO DB] for username='user_41'


[EXTRACTED] relation='favorite_movie', object='horror', topic='Movie', task='Content Consumption'

INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
    "response": "It's great to know that you enjoy horror movies! If you're looking for recommendations in this genre, feel free to ask.",
    "reason": "Acknowledging the user's preference for horror movies and offering assistance in finding recommendations.",
    "used_tool": "PersonaExtractor"
}
I see that you enjoy horror movies! If you're looking for recommendations in this genre, feel free to ask.

> Finished chain.
{
    "response": "It's great to know that you enjoy horror movies! If you're looking for recommendations in this genre, feel free to ask.",
    "reason": "Acknowledging the user's preference for horror movies and offering assistance in finding recommendations.",
    "used_tool": "PersonaExtractor"
}
I see that you enjoy horror movies! If you're looking for recommendations in this genre, feel free to ask.


In [7]:
response = agent.handle_task("Can you provide community recommendations?")

print(response)



> Entering new AgentExecutor chain...


INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO - [TOOL INVOKED] Community recommender triggered with input: Can you provide community recommendations?



Invoking: `CommunityRecommender` with `{'sentence': 'Can you provide community recommendations?'}`


[TOOL RESULT]
Type: Recommendation:
Community suggestions for Content Consumption (from other users):
- Book:
  . reading graphic novels: liked by 5 users
  . reading science fiction: liked by 5 users
  . reading cookbooks: liked by 4 users
- Game:
  . card games: liked by 6 users
  . strategy games: liked by 6 users
  . arcade games: liked by 5 users
- Movie:
  . watching animated movies: liked by 6 users
  . watching fantasy movies: liked by 6 users
  . watching historical dramas: liked by 6 users
- Music:
  . hip hop music: liked by 6 users
  . indie music: liked by 6 users
  . reggae music: liked by 6 users

INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
    "response": "Here are some community recommendations for Content Consumption:",
    "reason": "Providing community-based recommendations based on what other users prefer.",
    "used_tool": "CommunityRecommender"
}

> Finished chain.
{
    "response": "Here are some community recommendations for Content Consumption:",
    "reason": "Providing community-based recommendations based on what other users prefer.",
    "used_tool": "CommunityRecommender"
}


In [8]:
# Simple invocation - just pass the task
response = agent.handle_task("So, which movies should I watch tonight?")

print(response)



> Entering new AgentExecutor chain...


INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO - [TOOL INVOKED] Community recommender triggered with input: Which movies should I watch tonight?



Invoking: `CommunityRecommender` with `{'sentence': 'Which movies should I watch tonight?'}`


[TOOL RESULT]
Type: Recommendation:
Community suggestions for Content Consumption (from other users):
- Book:
  . reading graphic novels: liked by 5 users
  . reading science fiction: liked by 5 users
  . reading cookbooks: liked by 4 users
- Game:
  . card games: liked by 6 users
  . strategy games: liked by 6 users
  . arcade games: liked by 5 users
- Movie:
  . watching animated movies: liked by 6 users
  . watching fantasy movies: liked by 6 users
  . watching historical dramas: liked by 6 users
- Music:
  . hip hop music: liked by 6 users
  . indie music: liked by 6 users
  . reggae music: liked by 6 users

INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
    "response": "You should consider watching animated movies, fantasy movies, or historical dramas tonight.",
    "reason": "Based on community recommendations for movies, these genres are popular among other users.",
    "used_tool": "CommunityRecommender"
}

> Finished chain.
{
    "response": "You should consider watching animated movies, fantasy movies, or historical dramas tonight.",
    "reason": "Based on community recommendations for movies, these genres are popular among other users.",
    "used_tool": "CommunityRecommender"
}
